# Implementing a neural network in Python

---

## Programming Environment

1

```bash
conda create -n tf -v -y \
-c conda-forge --strict-channel-priority \
'imbalanced-learn=0.11' \
'matplotlib      =3.07' \
'numpy           =1.25' \
'pandas          =2.01' \
'plotly          =5.16' \
'python          =3.11' \
'scikit-learn    =1.03'
```

2

```bash
conda activate tf
```

3

TensorFlow latest

```bash
python -m pip install tensorflow
```

TensorFlow 2.12

```bash
python -m pip install tensorflow-macos==2.12
```

4

[ [d](https://developer.apple.com/metal/tensorflow-plugin/) ][ [p](https://pypi.org/project/tensorflow-metal/) ][ [f](https://developer.apple.com/forums/tags/tensorflow-metal/) ] tensorflow-metal latest

```bash
python -m pip install tensorflow-metal
```

tensorflow-metal 0.8

```bash
python -m pip install tensorflow-metal==0.8
```

In [3]:
import numpy  as np
np.set_printoptions(suppress=True) # suppress scientific notation
import pandas as pd

import sklearn
from   sklearn.model_selection import train_test_split
from   sklearn.preprocessing   import StandardScaler

import tensorflow as tf

import datetime
from   importlib.metadata import version
import os
import platform as p
import sys

pad = 20
print(  f"\n{'Executed' : <{pad}} : {datetime.datetime.now().astimezone().strftime('%Y-%m-%d %H:%M:%S %z %Z')}"
        f"\n{'Platform' : <{pad}} : {p.platform(aliased = False, terse = False)}"
        f"\n{'Conda'    : <{pad}} : {os.environ['CONDA_DEFAULT_ENV'] or sys.executable.split('/')[-3]}"
        f"\n{'Python'   : <{pad}} : {p.python_implementation()} {p.python_version()} {sys.executable}")
print(*[f'{name : <{pad}} : {version(name)}'
        for name in ['NumPy', 'Pandas', 'Scikit-Learn', 'TensorFlow-macos', 'TensorFlow-metal']], sep = '\n')


Executed             : 2023-09-03 18:47:16 -0400 EDT
Platform             : macOS-13.5.1-arm64-arm-64bit
Conda                : ml
Python               : CPython 3.11.5 /Users/df/anaconda3/envs/ml/bin/python
NumPy                : 1.23.5
Pandas               : 2.1.0
Scikit-Learn         : 1.3.0
TensorFlow-macos     : 2.12.0
TensorFlow-metal     : 0.8.0


---

In [4]:
# application_df = pd.read_csv(filepath_or_buffer = 'https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv')
# application_df.head()

In [5]:
# application_df.to_csv(path_or_buf = 'charity_data.csv',
#                       index       = False)

---

In [6]:
application_df = pd.read_csv(filepath_or_buffer = 'charity_data.csv')
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [7]:
application_df = application_df.drop(labels = ['EIN', 'NAME'],
                                     axis   = 1)
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [9]:
application_types_to_replace = list(application_counts[application_counts < 500].index)
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [10]:
for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(to_replace = app, value = 'Other')

application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [11]:
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [12]:
class_counts[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [13]:
class_counts[class_counts > 10]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: count, dtype: int64

In [14]:
classifications_to_replace = list(class_counts[class_counts < 1000].index)
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [15]:
for cls in classifications_to_replace:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(to_replace = cls, value = 'Other')

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [16]:
application_with_dummies_df = pd.get_dummies(data = application_df)
application_with_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


In [17]:
X = application_with_dummies_df.drop(labels = ['IS_SUCCESSFUL'], axis = 'columns').values
y = application_with_dummies_df['IS_SUCCESSFUL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78)

print(X      .shape)
print(y      .shape)
print(X_train.shape)
print(X_test .shape)
print(y_train.shape)
print(y_test .shape)

(34299, 43)
(34299,)
(25724, 43)
(8575, 43)
(25724,)
(8575,)


In [18]:
scaler = StandardScaler()
scaler

StandardScaler()

In [19]:
X_scaler = scaler.fit(X = X_train)
X_scaler

StandardScaler()

In [20]:
X_train_scaled = X_scaler.transform(X = X_train)
X_train_scaled.shape

(25724, 43)

In [21]:
X_test_scaled = X_scaler.transform(X = X_test)
X_test_scaled.shape

(8575, 43)

In [22]:
number_input_features = len(X_train[0]) # 43
hidden_nodes_layer1   = 80
hidden_nodes_layer2   = 30

nn = tf.keras.models.Sequential()
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer1,
                                     input_dim  = number_input_features,
                                     activation = 'relu'))
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer2,
                                     activation = 'relu'))
nn.add(layer = tf.keras.layers.Dense(units      = 1,
                                     activation = 'sigmoid'))
nn.summary()

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [23]:
nn.compile(optimizer = 'adam',
           loss      = 'binary_crossentropy',
           metrics   = ['accuracy'])

nn.fit(x       = X_train_scaled,
       y       = y_train,
       epochs  = 100)

Epoch 1/100


2023-09-03 18:47:38.391491: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


804/804 [==============================] - 6s 7ms/step - loss: 0.5986 - accuracy: 0.7142
Epoch 2/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6007 - accuracy: 0.7121
Epoch 3/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6160 - accuracy: 0.7037
Epoch 4/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6320 - accuracy: 0.6920
Epoch 5/100
804/804 [==============================] - 6s 8ms/step - loss: 0.6834 - accuracy: 0.6791
Epoch 6/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6999 - accuracy: 0.6742
Epoch 7/100
804/804 [==============================] - 6s 7ms/step - loss: 0.7398 - accuracy: 0.6666
Epoch 8/100
804/804 [==============================] - 6s 7ms/step - loss: 0.7564 - accuracy: 0.6533
Epoch 9/100
804/804 [==============================] - 6s 7ms/step - loss: 0.7664 - accuracy: 0.6557
Epoch 10/100
804/804 [==============================] - 6s 7ms/step - loss: 0.8566 - accuracy: 0.6497
E

In [24]:
model_loss, model_accuracy = nn.evaluate(x = X_test_scaled,
                                         y = y_test,
                                         verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 4.3818 - accuracy: 0.7090 - 1s/epoch - 5ms/step
Loss: 4.381792068481445, Accuracy: 0.7090378999710083


In [25]:
nn.save(filepath = 'AlphabetSoupCharity.keras')
#nn.save('AlphabetSoupCharity.h5')

---

In [26]:
application_df = pd.read_csv(filepath_or_buffer = 'charity_data.csv')
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [27]:
application_df = application_df.drop(labels = ['EIN'],
                                     axis   = 1)
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [28]:
name_counts = application_df['NAME'].value_counts()
name_counts[name_counts > 1]

NAME
PARENT BOOSTER USA INC                                            1260
TOPS CLUB INC                                                      765
UNITED STATES BOWLING CONGRESS INC                                 700
WASHINGTON STATE UNIVERSITY                                        492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                    408
                                                                  ... 
VETERANS OF FOREIGH WARS OF THE UNITED STATES DEPT OF COLORADO       2
ETA PHI BETA SORORITY INC                                            2
POINT MAN INTERNATIONAL MINISTRIES                                   2
MULTI COMMUNITY DIVERSIFIED SERVICES INC                             2
VETERANS OF FOREIGN WARS AUXILIARY                                   2
Name: count, Length: 792, dtype: int64

In [29]:
names_to_replace = list(name_counts[name_counts <= 5].index)
names_to_replace

['FLORIDA FAMILY CHILD CARE HOME ASSOCIATION INC',
 'GERONTOLOGICAL ADVANCED PRACTICE NURSES ASSOCIATION',
 'INTERNATIONL TRANSPLANT NURSES SOCIETY',
 'NATIONAL ORGANIZATION FOR WOMEN INC',
 'PTA HAWAII CONGRESS',
 'HIGH TWELVE INTERNATIONAL',
 'CHILDRENS HEART FOUNDATION',
 'INTERNATIONAL ORDER OF THE RAINBOW GIRLS THE SUPREME ASSEMBLY',
 'UNITED ASSOCIATION OF JOURNEYMEN & APPRENTICES OF THE PLUMBING & PIPE',
 'WARRIOR BROTHERHOOD OF VETERANS MOTORCYCLE CLUB INC',
 'CHC COMMUNITY CARE LLC',
 'DEPARTMENT OF NEVADA VETERANS OF FOREIGN WARS OF THE US INC',
 'RISK MANAGEMENT ASSOCIATES FLORIDA CHAPTER CORP',
 'NATIONAL ASSOCIATION OF WATCH AND CLOCK COLLECTORS INC',
 'HARMONY INC',
 'VETERANS OF FOREIGN WARS OF THE UNITED STATES DEPT OF COLORADO',
 'AMERICAN-TURKISH ASSOCIATION OF SOUTHERN CALIFORNIA',
 'NATIONAL ASSOCIATION OF COMPOSERS USA',
 'HERITAGE ACADEMY INC',
 'COUNTRY DANCE AND SONG SOCIETY INC',
 'NATIONAL HUGUENOT SOCIETY',
 'FATHER FLANAGANS BOYS HOME',
 'VETERANS OF FOREIGN

In [30]:
for app in names_to_replace:
  application_df['NAME'] = application_df['NAME'].replace(to_replace = app, value = 'Other')

application_df['NAME'].value_counts()

NAME
Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: count, Length: 355, dtype: int64

In [31]:
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [32]:
application_types_to_replace = list(application_counts[application_counts < 500].index)
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [33]:
for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(to_replace = app, value = 'Other')

application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [34]:
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [35]:
classes_to_replace = list(class_counts[class_counts < 1_000].index)
classes_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [36]:
for cls in classes_to_replace:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(to_replace = cls,
                                                                              value      = 'Other')

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [37]:
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [38]:
application_with_dummies_df = pd.get_dummies(data = application_df)
application_with_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [39]:
X = application_with_dummies_df.drop(labels = ['IS_SUCCESSFUL'],
                                     axis   = 'columns').values
y = application_with_dummies_df['IS_SUCCESSFUL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [40]:
scaler = StandardScaler()
scaler

StandardScaler()

In [41]:
X_scaler = scaler.fit(X = X_train)
X_scaler

StandardScaler()

In [42]:
X_train_scaled = X_scaler.transform(X = X_train)
X_train_scaled

array([[ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       ...,
       [ 0.01394306, -0.03229417, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.01093629, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425]])

In [43]:
X_test_scaled = X_scaler.transform(X = X_test)
X_test_scaled

array([[ 0.01394306, -0.03336766, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306,  0.02868029, -0.01527415, ..., 14.31055554,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       ...,
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.06987849,
         0.02789425, -0.02789425]])

In [44]:
number_input_features = len(X_train[0]) # 398
hidden_nodes_layer1   = 100
hidden_nodes_layer2   =  30
hidden_nodes_layer3   =  10

nn = tf.keras.models.Sequential()
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer1,
                                     input_dim  = number_input_features,
                                     activation = 'relu'))
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer2,
                                     activation = 'sigmoid'))
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer3,
                                     activation = 'sigmoid'))
nn.add(layer = tf.keras.layers.Dense(units      = 1,
                                     activation = 'sigmoid'))
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               39900     
                                                                 
 dense_4 (Dense)             (None, 30)                3030      
                                                                 
 dense_5 (Dense)             (None, 10)                310       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43,251
Trainable params: 43,251
Non-trainable params: 0
_________________________________________________________________


In [45]:
nn.compile(optimizer = 'adam',
           loss      = 'binary_crossentropy',
           metrics   = ['accuracy'])

fit_model = nn.fit(x = X_train_scaled,
                   y = y_train,
                   epochs = 100)

Epoch 1/100
804/804 [==============================] - 8s 9ms/step - loss: 0.4963 - accuracy: 0.7690
Epoch 2/100
804/804 [==============================] - 7s 8ms/step - loss: 0.4452 - accuracy: 0.7904
Epoch 3/100
804/804 [==============================] - 7s 8ms/step - loss: 0.4372 - accuracy: 0.7936
Epoch 4/100
804/804 [==============================] - 7s 8ms/step - loss: 0.4329 - accuracy: 0.7956
Epoch 5/100
804/804 [==============================] - 6s 8ms/step - loss: 0.4300 - accuracy: 0.7949
Epoch 6/100
804/804 [==============================] - 6s 8ms/step - loss: 0.4278 - accuracy: 0.7970
Epoch 7/100
804/804 [==============================] - 6s 8ms/step - loss: 0.4259 - accuracy: 0.7972
Epoch 8/100
804/804 [==============================] - 6s 8ms/step - loss: 0.4241 - accuracy: 0.7984
Epoch 9/100
804/804 [==============================] - 6s 8ms/step - loss: 0.4230 - accuracy: 0.7984
Epoch 10/100
804/804 [==============================] - 6s 8ms/step - loss: 0.4218 - accura

In [ ]:
model_loss, model_accuracy = nn.evaluate(x = X_test_scaled,
                                         y = y_test,
                                         verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

: 

: 

: 

: 

: 

: 

In [ ]:
nn.save(filepath = 'AlphabetSoupCharityOptimization.keras')
#nn.save('AlphabetSoupCharityOptimization.h5')

: 

: 

: 

: 

: 

: 

---

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 128,
                                  random_state = 78)
rf_model

: 

: 

: 

: 

: 

: 

In [ ]:
rf_model = rf_model.fit()

: 

: 

: 

: 

: 

: 

In [ ]:
y_pred = rf_model.predict(X_test_scaled)
print(f'Random forest model accuracy: {accuracy_score(y_test, y_pred):.3f}')

: 

: 

: 

: 

: 

: 

---